### Import Statements

In [1]:
import os
import cv2
import pylab
import dlib
import numpy as np
import pandas as pd
#import imageio
import skimage.io as imageio
import sys
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
#from keras.utils import multi_gpu_model
from keras.optimizers import SGD, RMSprop
from keras.layers import Concatenate, Input, concatenate, add, multiply, maximum
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, generic_utils
from sklearn.model_selection import train_test_split
#from sklearn import cross_validation
from sklearn import preprocessing
from keras import backend as K
import matplotlib.pyplot as plt
%matplotlib inline
#K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')
#import matplotlib
#matplotlib.use('Agg')

### GPU Check

In [2]:
gpu_available = tf.config.list_physical_devices('GPU')

In [3]:
gpu_available

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
# DLib Face Detection
predictor_path = "/Thesis/MicroExpSTCNN/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [6]:
def get_landmark(img):
    rects = detector(img, 1)
    if len(rects) > 1:
        pass
    if len(rects) == 0:
        pass
    ans = np.matrix([[p.x, p.y] for p in predictor(img, rects[0]).parts()])
    return ans

In [7]:
def annotate_landmarks(img, landmarks, font_scale = 0.4):
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(img, str(idx), pos, fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, fontScale=font_scale, color=(0, 0, 255))
        cv2.circle(img, pos, 3, color=(0, 255, 255))
    return img


In [ ]:
new_path = '/Thesis/Sample_dataset/006/006_1_2/006_05626.jpg'

In [ ]:
image_new = cv2.imread(new_path)

In [ ]:
imageio.imshow(image_new)

In [ ]:
landmarks = get_landmark(image_new)

In [ ]:
numpylandmarks = np.asarray(landmarks)
eye_image = image_new[numpylandmarks[38][1]-60:numpylandmarks[41][1]+30, numpylandmarks[36][0]-50:numpylandmarks[39][0]+30]

In [ ]:
numpylandmarks = np.asarray(landmarks)
face = image_new[numpylandmarks[20][1]-40:numpylandmarks[9][1], numpylandmarks[0][0]+30:numpylandmarks[16][0]-30]

In [ ]:
imageio.imshow(face)

In [ ]:
new_landmark = np.asarray(landmarks)

In [ ]:
eye_image.shape

In [ ]:
eye_image = cv2.resize(eye_image, (32, 32), interpolation = cv2.INTER_AREA)
eye_image = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)

In [ ]:
imageio.imshow(eye_image)

### Loading Excel to DataFrame

In [8]:
df = pd.read_excel('/Thesis/Datasets/CASME2/CASME2-coding-20190701.xlsx') #index_col=0

In [ ]:
df.count()

In [ ]:
df.head()

In [ ]:
df['Estimated Emotion'].unique()

In [ ]:
df['Subject'].unique()

In [ ]:
df['Estimated Emotion'].value_counts()

In [9]:
#positive = df[df['Estimated Emotion'] == 'happiness'][['Subject','Filename']]
#print(positive)
df['Subject'] = df['Subject'].astype(str)
df['Sub_FileName'] = df[['Subject', 'Filename']].apply(lambda x: '_'.join(x), axis=1)
df.head(5)

,Subject,Filename,Unnamed: 2,OnsetFrame,ApexFrame,OffsetFrame,Unnamed: 6,Action Units,Estimated Emotion,Sub_FileName
0,1,EP02_01f,NaN,46,59,86,NaN,12,happiness,1_EP02_01f
1,1,EP03_02,NaN,131,139,161,NaN,18,others,1_EP03_02
2,1,EP04_02,NaN,21,54,76,NaN,4,others,1_EP04_02
3,1,EP04_03,NaN,31,41,56,NaN,4,others,1_EP04_03
4,1,EP04_04,NaN,23,49,66,NaN,4,others,1_EP04_04


In [10]:
positive = df[df['Estimated Emotion'] == 'happiness']['Sub_FileName']#[['Subject','Filename']]
print('Positive :', positive.count())

negative = df[(df['Estimated Emotion'] == 'disgust') | (df['Estimated Emotion'] == 'repression') | (df['Estimated Emotion'] == 'fear') | (df['Estimated Emotion'] == 'sadness')]['Sub_FileName']
print('Negative :',negative.count())

surprise = df[df['Estimated Emotion'] == 'surprise']['Sub_FileName']
print('Surprise :',surprise.count())

Positive : 32
Negative : 96
Surprise : 28


In [ ]:
df['Sub_FileName'].str.split('_')[1][0]

In [ ]:
positive = positive.str.split('_')
positive[1]

In [ ]:
positive = positive.str.split('_')
positive = positive.tolist()
positive

In [ ]:
df['Sub_FileName'].str.split('_')[9][0]

### Initialization

In [ ]:
subjectPath = '/Thesis/Datasets/CASME2/CASME2_RAW_selected/'

In [ ]:
image_rows, image_columns, frames_Count = 64, 64, 50
training_samm_list = []

In [ ]:
i, j, k = 0, 0, 0

eye_training_list = []
nose_training_list = []

In [ ]:
for lab in [positive, surprise]: # negative,
    subDirectory = lab#.tolist() 
    count = 0
    for sub in subDirectory:
        #labelpath = '/Thesis/Datasets/SAMM/'+ sub[:3] +'/'# + sub + '/'
        #print(sub.str.split('_'))
        #print(sub)
        sub = str(sub).split('_')
        #type(sub)
        #print(sub)
        #print(sub[0].zfill(2))
        #print(sub[1]+'_'+sub[2])
        #print(sub.split('_'))

In [ ]:
i, j, k = 0, 0, 0

eye_training_list = []
nose_training_list = []

In [ ]:
sub_count = 0
for lab in [positive, negative, surprise]: 
    subDirectory = lab#.tolist() 
    count = 0
    for sub in subDirectory:
        sub = str(sub).split('_')
        labelpath = '/Thesis/Datasets/CASME2/CASME2_RAW_selected/'+'sub'+ sub[0].zfill(2) +'/'
        directorylisting = os.listdir(labelpath)
        #print('searching based: ', (sub[1]+'_'+sub[2]))
        #print('Existing dir: ',directorylisting[1])
    #break
        for video in directorylisting:
            if video == str(sub[1]+'_'+sub[2]):
                #print('Matched dir: ',dirList)
                videopath = labelpath + video
                frames = []
                eye_frames = []
                nose_mouth_frames = []
                framelisting = os.listdir(videopath)
                #loadedvideo = Im.get_reader(videopath, format='FFMPEG' )#, 'ffmpeg')
                if len(framelisting) > frames_Count:
                    framerange = [x  for x in range(frames_Count)]
                    count = count + 1
                    for frame in framerange:
                        #image = loadedvideo.get_data(frame)
                        imagepath = videopath + "/" + framelisting[frame]
                        image = cv2.imread(imagepath)
                    cv2.imshow(image)
                        #break
            break
        break
                    
                    #     landmarks = get_landmark(image)
                    #     numpylandmarks = np.asarray(landmarks)
                    #     eye_image = image[numpylandmarks[19][1]:numpylandmarks[1][1], numpylandmarks[1][0]:numpylandmarks[15][0]]
                    #     eye_image = cv2.resize(eye_image, (32, 32), interpolation = cv2.INTER_AREA)
                    #     eye_image = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)
                    #     nose_mouth_image = image[numpylandmarks[2][1]:numpylandmarks[6][1], numpylandmarks[2][0]:numpylandmarks[14][0]]
                    #     nose_mouth_image = cv2.resize(nose_mouth_image, (32, 32), interpolation = cv2.INTER_AREA)
                    #     nose_mouth_image = cv2.cvtColor(nose_mouth_image, cv2.COLOR_BGR2GRAY)
                    #     eye_frames.append(eye_image)
                    #     nose_mouth_frames.append(nose_mouth_image)
                    # eye_frames = np.asarray(eye_frames)
                    # nose_mouth_frames = np.asarray(nose_mouth_frames)

### CASME-II Dataset

In [ ]:
sub_count = 0
for lab in [positive, negative, surprise]: 
    subDirectory = lab#.tolist() 
    count = 0
    for sub in subDirectory:
        sub = str(sub).split('_')
        labelpath = '/Thesis/Datasets/CASME2/CASME2_RAW_selected/'+'sub'+ sub[0].zfill(2) +'/'
        directorylisting = os.listdir(labelpath)
        #print('searching based: ', (sub[1]+'_'+sub[2]))
        #print('Existing dir: ',directorylisting[1])
    #break
        for video in directorylisting:
            if video == str(sub[1]+'_'+sub[2]):
                #print('Matched dir: ',dirList)
                videopath = labelpath + video
                frames = []
                eye_frames = []
                nose_mouth_frames = []
                framelisting = os.listdir(videopath)
                #loadedvideo = Im.get_reader(videopath, format='FFMPEG' )#, 'ffmpeg')
                if len(framelisting) > frames_Count:
                    framerange = [x  for x in range(frames_Count)]
                    count = count + 1
                    for frame in framerange:
                        #image = loadedvideo.get_data(frame)
                        imagepath = videopath + "/" + framelisting[frame]
                        image = cv2.imread(imagepath)
                        landmarks = get_landmark(image)
                        numpylandmarks = np.asarray(landmarks)
                        eye_image = image[numpylandmarks[19][1]:numpylandmarks[1][1], numpylandmarks[1][0]:numpylandmarks[15][0]]
                        eye_image = cv2.resize(eye_image, (32, 32), interpolation = cv2.INTER_AREA)
                        eye_image = cv2.cvtColor(eye_image, cv2.COLOR_BGR2GRAY)
                        nose_mouth_image = image[numpylandmarks[2][1]:numpylandmarks[6][1], numpylandmarks[2][0]:numpylandmarks[14][0]]
                        nose_mouth_image = cv2.resize(nose_mouth_image, (32, 32), interpolation = cv2.INTER_AREA)
                        nose_mouth_image = cv2.cvtColor(nose_mouth_image, cv2.COLOR_BGR2GRAY)
                        eye_frames.append(eye_image)
                        nose_mouth_frames.append(nose_mouth_image)
                    eye_frames = np.asarray(eye_frames)
                    nose_mouth_frames = np.asarray(nose_mouth_frames)
                    eye_videoarray = np.rollaxis(np.rollaxis(eye_frames, 2, 0), 2, 0)
                    nose_mouth_videoarray = np.rollaxis(np.rollaxis(nose_mouth_frames, 2, 0), 2, 0)
                    eye_training_list.append(eye_videoarray)
                    nose_training_list.append(nose_mouth_videoarray)

    print(count)
    # break

In [ ]:
eye_training_list = np.asarray(eye_training_list)
nose_training_list = np.asarray(nose_training_list)

eye_trainingsamples = len(eye_training_list)
nose_trainingsamples = len(nose_training_list)

eye_traininglabels = np.zeros((eye_trainingsamples, ), dtype = int)
nose_traininglabels = np.zeros((nose_trainingsamples, ), dtype = int)

In [ ]:
27+75+20


In [ ]:
eye_traininglabels[0:27] = 0
eye_traininglabels[27:102] = 1
eye_traininglabels[102:122] = 2

nose_traininglabels[0:27] = 0
nose_traininglabels[27:102] = 1
nose_traininglabels[102:122] = 2

In [ ]:
eye_traininglabels = np_utils.to_categorical(eye_traininglabels, 3)
nose_traininglabels = np_utils.to_categorical(nose_traininglabels, 3)

In [ ]:
etraining_data = [eye_training_list, eye_traininglabels]
(etrainingframes, etraininglabels) = (etraining_data[0], etraining_data[1])
etraining_set = np.zeros((eye_trainingsamples, 1, 32, 32, 50))
for h in range(eye_trainingsamples):
    etraining_set[h][0][:][:][:] = etrainingframes[h,:,:,:]

In [ ]:
etraining_set = etraining_set.astype('float32')
etraining_set -= np.mean(etraining_set)
etraining_set /= np.max(etraining_set)

In [ ]:
ntraining_data = [nose_training_list, nose_traininglabels]
(ntrainingframes, ntraininglabels) = (ntraining_data[0], ntraining_data[1])
ntraining_set = np.zeros((nose_trainingsamples, 1, 32, 32, 50))
for h in range(nose_trainingsamples):
        ntraining_set[h][0][:][:][:] = ntrainingframes[h,:,:,:]

In [ ]:
ntraining_set = ntraining_set.astype('float32')
ntraining_set -= np.mean(ntraining_set)
ntraining_set /= np.max(ntraining_set)

In [ ]:
np.save('/Thesis/MicroExpSTCNN/Training_dataset_CASME/intermediate_microexpfusenetnoseimages.npy', ntraining_set)
np.save('/Thesis/MicroExpSTCNN/Training_dataset_CASME/intermediate_microexpfuseneteyeimages.npy', etraining_set)
np.save('/Thesis/MicroExpSTCNN/Training_dataset_CASME/intermediate_microexpfusenetnoselabels.npy', nose_traininglabels)
np.save('/Thesis/MicroExpSTCNN/Training_dataset_CASME/intermediate_microexpfuseneteyelabels.npy', eye_traininglabels)

In [ ]:
# Load training images and labels that are stored in numpy array

ntraining_set = np.load('/Thesis/MicroExpSTCNN/Training_dataset_CASME/intermediate_microexpfusenetnoseimages.npy')
etraining_set = np.load('/Thesis/MicroExpSTCNN/Training_dataset_CASME/intermediate_microexpfuseneteyeimages.npy')
eye_traininglabels = np.load('/Thesis/MicroExpSTCNN/Training_dataset_CASME/intermediate_microexpfusenetnoselabels.npy')
nose_traininglabels = np.load('/Thesis/MicroExpSTCNN/Training_dataset_CASME/intermediate_microexpfuseneteyelabels.npy')

In [ ]:
# Intermediate MicroExpFuseNet Model
eye_input = Input(shape = (1, 32, 32, 50))
eye_conv = Convolution3D(32, (3, 3, 15))(eye_input)
ract_1 = Activation('relu')(eye_conv)
eye_max = MaxPooling3D(pool_size=(3, 3, 3))(ract_1)
dropout_1 = Dropout(0.5)(eye_max)
eye_flatten = Flatten()(dropout_1)

nose_input = Input(shape = (1, 32, 32, 50))
nose_conv = Convolution3D(32, (3, 3, 15))(nose_input)
ract_2 = Activation('relu')(nose_conv)
nose_max = MaxPooling3D(pool_size=(3, 3, 3))(ract_2)
dropout_2 = Dropout(0.5)(nose_max)
nose_flatten = Flatten()(dropout_2)

result = Concatenate(axis = 1)([eye_flatten, nose_flatten])
dense_1 = Dense(1024, )(result)
dropout_4 = Dropout(0.5)(dense_1)
dense_2 = Dense(128, )(dropout_4)
dropout_5 = Dropout(0.5)(dense_2)
dense_3 = Dense(3, )(dropout_5)
activation = Activation('softmax')(dense_3)

model = Model(inputs = [eye_input, nose_input], outputs = activation)

model.compile(loss = 'categorical_crossentropy', optimizer = 'SGD', metrics = ['accuracy'])

filepath="weights_IF/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.summary()

In [ ]:
# Load pre-trained weights
#model.load_weights('weights_intermediate_microexpfusenet/weights-improvement-22-0.83.hdf5')

In [ ]:
# Spliting the dataset into training and validation sets
etrain_images, evalidation_images, etrain_labels, evalidation_labels =  train_test_split(etraining_set, eye_traininglabels, test_size=0.2, shuffle=False)
ntrain_images, nvalidation_images, ntrain_labels, nvalidation_labels =  train_test_split(ntraining_set, nose_traininglabels, test_size=0.2, shuffle=False)

In [ ]:
# Save validation set in a numpy array
np.save('/Thesis/MicroExpSTCNN/Eval_dataset_CASME/intermediate_microexpfusenet_eval_images.npy', evalidation_images)
np.save('/Thesis/MicroExpSTCNN/Eval_dataset_CASME/intermediate_microexpfusenet_nval_images.npy', nvalidation_images)
np.save('/Thesis/MicroExpSTCNN/Eval_dataset_CASME/intermediate_microexpfusenet_eval_labels.npy', evalidation_labels)
np.save('/Thesis/MicroExpSTCNN/Eval_dataset_CASME/intermediate_microexpfusenet_nval_labels.npy', nvalidation_labels)

In [ ]:
# Training the model
history = model.fit([etrain_images, ntrain_images], etrain_labels, validation_data = ([evalidation_images, nvalidation_images], evalidation_labels), callbacks=callbacks_list, batch_size = 16, epochs = 100, shuffle=True)

In [ ]:
# Loading Load validation set from numpy array
eimg = np.load('/Thesis/MicroExpSTCNN/Eval_dataset_CASME/intermediate_microexpfusenet_eval_images.npy')
nimg = np.load('/Thesis/MicroExpSTCNN/Eval_dataset_CASME/intermediate_microexpfusenet_nval_images.npy')
labels = np.load('/Thesis/MicroExpSTCNN/Eval_dataset_CASME/intermediate_microexpfusenet_eval_labels.npy')

In [ ]:
# Finding Confusion Matrix using pretrained weights
predictions = model.predict([eimg, nimg])
predictions_labels = np.argmax(predictions, axis=1)
validation_labels = np.argmax(labels, axis=1)
cfm = confusion_matrix(validation_labels, predictions_labels)
print (cfm)